Import necessary libraries, and define our schemata.

In [ ]:
from fastscore import ipmagic
import pandas as pd

sch_in = '{"type":"record", "name":"xy", "fields":[{"name":"x", "type":"double"}, {"name":"y", "type":"double"}]}'
sch_out = '{"type":"record", "name":"xyz", "fields":[{"name":"x", "type":"double"}, {"name":"y", "type":"double"}, {"name":"z", "type":"double"}]}'

Optionally, we can create "attachments"---external files used by the model. Here, we just pickle an object:

In [ ]:
import pickle

model_params = {'a': 1, 'b': 2}
pickle.dump(model_params, open('model_params.pkl', 'wb'))

## Model Definition

Use the `py3model` cell magic to define a model (Python notebooks only)

In [ ]:
%%py3model
 
# fastscore.input: sch_in
# fastscore.output: sch_out
# fastscore.recordsets: none
 
import numpy as np
import pandas as pd
import pickle
 
def begin():
    global model_params
    model_params = pickle.load(open('model_params.pkl', 'rb'))
 
def action(datum):
    mydf = datum
    mydf['z'] = model_params['a']*mydf['x'] + model_params['b']*mydf['y']
    # yield dict(mydf.sum()) # recordset input, regular output
    yield mydf # recordsets input and output
    return

score with JSON serialization...

In [ ]:
_model.score(['{"x":1.0, "y":2.0}'], use_json = True)

or without JSON serialization:

In [ ]:
_model.score([{"x":1.0, "y":2.0}], use_json = False)

Model validation tests that inputs and desired outputs conform to schema, and then compares scores generated to the expected results. JSON serialization of inputs and outputs is optional.

In [ ]:
_model.validate(['{"x":1.0, "y":2.0}'], ['{"y": 2.0, "x": 1.0, "z": 3.0}'], use_json = True)

Record sets are supported.

In [ ]:
# now we use dataframes
_model.options['recordsets'] = 'both' # switch to using record sets

# create a DataFrame
mydf = pd.DataFrame({'x':[1.0, 2.0, 3.0], 'y':[4.0, 5.0, 6.0]})
_model.score(mydf)

In [ ]:
# or, use JSON-serialized inputs
from fastscore.codec import to_json

inputs = [x for x in to_json(mydf, _model.input_schema)]
_model.score(inputs, use_json = True)

Rebind the model's `action` method or other functions.

In [ ]:
def action(datum):
    datum['z'] = model_params['a']*datum['x'] - model_params['b']*datum['y']
    yield datum

_model.action = action

See what the model will look like by exporting to a string:

In [ ]:
print(_model.to_string())

Now let's add the attachment we created. You can bundle multiple external files into a single attachment. FastScore expects an attachment to be either a gzipped tarball (.tar.gz) or just a .zip file.

In [ ]:
import tarfile

with tarfile.open('attachment.tar.gz', 'w:gz') as tar:
    tar.add('model_params.pkl')

_model.attachments.append('attachment.tar.gz')

# FastScore Engines

The Engine class lets you interact directly with a running engine.

In [ ]:
# now we connect to a running FastScore engine:
from fastscore import Engine
engine = Engine('https://localhost:8000', container='engine-x-2')

`Model` objects can be deployed directly to an `Engine`. (Check the FastScore Dashboard!)

In [ ]:
# Give our model a name, and deploy it.
_model.name = 'example_py3_model'
engine.deploy(_model)

Automatic JSON serialization lets you pass Python objects directly to a running `Engine` for scoring.

In [ ]:
mydf

In [ ]:
# score using either Python objects (in this case, a DataFrame)
engine.score(mydf)

Stop the `Engine` when you're done.

In [ ]:
engine.stop()